## 🚀 Best Local Open-Source Vector Databases (Vector Stores)

| **Vector DB**       | **Size**     | **Features**                                                                 | **Language**     | **Persistence**       | **Best For**                                 |
|---------------------|--------------|------------------------------------------------------------------------------|------------------|------------------------|-----------------------------------------------|
| **FAISS**           | ~50MB        | Fast, supports L2/IP, GPU, quantization, mature ecosystem                   | Python / C++     | ✅ Manual (save/load)  | Fast local similarity search                 |
| **Annoy**           | ~15MB        | Tree-based, fast, static, no training needed                                | Python           | ✅ Manual              | Small-to-medium fixed datasets               |
| **HNSWlib**         | ~10MB        | Graph-based (HNSW), accurate, very low memory use                           | Python / C++     | ✅ Manual              | Accurate local ANN search                    |
| **ScaNN**           | ~80MB        | Optimized for inner product / cosine, from Google                           | Python / C++     | ✅ Manual              | Cosine-heavy tasks, similarity search        |
| **Chroma (local)**  | ~100MB       | Built-in persistence, metadata support, LangChain-compatible                | Python           | ✅ Auto (persistent)   | RAG prototypes, local vector + metadata      |
| **Qdrant (local)**  | ~250MB       | Fast, rich filters, Docker or embedded mode                                 | Rust / Python    | ✅ Persistent          | Hybrid search (vector + metadata filters)    |
| **Weaviate (embedded)** | ~200MB   | REST & GraphQL, local embedded mode, metadata filtering                     | Python / Go      | ✅ Persistent          | Local + semantic + hybrid search             |
| **Milvus (lite)**   | ~500MB+      | Distributed, scalable, requires Docker or cloud                             | C++ / Go         | ✅ Persistent          | Large-scale production use                   |

---

### 📝 Notes:
- ✅ **Persistent** = Saves embeddings to disk and reloads after restart.
- Use **FAISS, HNSWlib, or Annoy** for lightweight, local-only projects.
- Use **Chroma, Qdrant, or Weaviate** for local RAG or metadata-rich search.

#

# 🧠 Vector Database Comparison: FAISS, Qdrant, Weaviate, and Chroma

This notebook covers:
- Storing embeddings using `all-MiniLM-L6-v2`
- Local use of FAISS, Qdrant, Weaviate, and Chroma
- Similarity/distance search
- Parametrized search types
- Interview questions & differences

---

## 📦 Prerequisites

```python
!pip install sentence-transformers faiss-cpu qdrant-client weaviate-client chromadb


## ✨ Generate Embeddings with all-MiniLM-L6-v2

In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

documents = ["Paris is the capital of France.",
             "Berlin is in Germany.", 
             "Tokyo is in Japan.", 
             "Tokyo is capital of Japan.",
             "kapil is gen ai engineer working for testing model",
             "kapil works in gen Ai area"]

embeddings = model.encode(documents,show_progress_bar=True,output_value='sentence_embedding')


Batches: 100%|██████████| 1/1 [00:00<00:00, 11.57it/s]


In [3]:
#vector size for each sentense
embeddings[0].size

384

## 🧠 FAISS: Facebook AI Similarity Search


In [4]:
import faiss
import numpy as np

embeddings_np = np.array(embeddings).astype('float32')
index = faiss.IndexFlatL2(embeddings_np.shape[1])
index.add(embeddings_np)


D, I = index.search(embeddings_np[:2], k=2)
print("Indices:\n", I)
print("Distances:\n", D)


Indices:
 [[0 1]
 [1 2]]
Distances:
 [[0.        1.4986992]
 [0.        1.1811731]]


In [5]:
#search nearest vector to query

query = "what is Tokyo"
embedded_query = model.encode([query])

embedded_query.size

distances, indices = index.search(embedded_query,k=2)

print("distance : ",distances)
print("indices : ",indices)

print("query : what is Tokyo")
print("Ans is : ->")
for i in indices[0]:
    print(documents[i])

distance :  [[0.29537767 0.31994855]]
indices :  [[3 2]]
query : what is Tokyo
Ans is : ->
Tokyo is capital of Japan.
Tokyo is in Japan.


## 📌 Qdrant: With Metadata and Filtering

In [6]:
import qdrant_client
from qdrant_client.models import VectorParams, Distance

client = qdrant_client.QdrantClient(path="qdrant_local")

client.create_collection(
    collection_name='docs',
    vectors_config=VectorParams(size=embeddings_np.shape[1], distance=Distance.COSINE)
)


True

In [22]:


payload = [{"text": doc} for doc in documents]

client.upload_collection(
    collection_name='docs',
    vectors=embeddings_np,
    payload=payload
)

query_vector = "what is capital"
embedded_query_vector = model.encode(query_vector)

results = client.query_points(
    collection_name='docs',
    query=embedded_query_vector,
    limit=3
)

for res in results:
    print(res[1][0].payload)


{'text': 'Tokyo is capital of Japan.'}


## 🌐 Weaviate: Graph + Schema

In [ ]:
# from weaviate import Client

# client = Client("http://localhost:8080")

# # Schema
# client.schema.delete_all()
# client.schema.create_class({
#     "class": "Document",
#     "vectorizer": "none",
#     "properties": [{"name": "text", "dataType": ["text"]}]
# })

# # Add
# for doc, emb in zip(documents, embeddings_np):
#     client.data_object.create({"text": doc}, "Document", vector=emb.tolist())

# # Search
# res = client.query.get("Document", ["text"])\
#     .with_near_vector({"vector": embeddings_np[0].tolist()})\
#     .with_limit(2).do()
# print(res)

import weaviate
print(weaviate.__file__)



## ⚡ Chroma: Lightweight & Pythonic


In [5]:
import chromadb
from chromadb.config import Settings

# --- Initialize persistent Chroma client ---
client = chromadb.Client(Settings(
    chroma_db_impl="duckdb+parquet",
    persist_directory="./chroma_data"
))

# --- Create or get a collection ---
collection_name = "docs"
collection = client.get_or_create_collection(name=collection_name)

# --- Add documents with embeddings ---
for idx, (doc, emb) in enumerate(zip(documents, embeddings_np)):
    collection.add(
        documents=[doc],
        metadatas=[{"text": doc}],
        ids=[str(idx)],
        embeddings=[emb.tolist()]
    )

# --- Perform a similarity query ---
query_vector = embeddings_np[0].tolist()
top_k = 2

results = collection.query(
    query_embeddings=[query_vector],
    n_results=top_k
)

# --- Display results ---
for i, (doc, score) in enumerate(zip(results["documents"][0], results["distances"][0])):
    print(f"Result {i+1}: {doc} (Score: {score:.4f})")


ValueError: [91mYou are using a deprecated configuration of Chroma.

[94mIf you do not have data you wish to migrate, you only need to change how you construct
your Chroma client. Please see the "New Clients" section of https://docs.trychroma.com/deployment/migration.
________________________________________________________________________________________________

If you do have data you wish to migrate, we have a migration tool you can use in order to
migrate your data to the new Chroma architecture.
Please `pip install chroma-migrate` and run `chroma-migrate` to migrate your data and then
change how you construct your Chroma client.

See https://docs.trychroma.com/deployment/migration for more information or join our discord at https://discord.gg/MMeYNTmh3x for help![0m